# Tugas Besar IF3170 - Inteligensi Buatan
# Aplikasi berbasis web untuk klasifikasi penyakit jantung berdasarkan data klinis terstruktur
## Tahap A - Eksperimen untuk mendapatkan model terbaik

***

## Dibuat oleh: Kelompok 3 - Unexpected
### Rizki Alif Salman Alfarisy / 13516005
### Jonathan Alvaro / 13516023
### Joseph Salimin / 13516037
### Kevin Leonardo Limitius / 13516049
### Kevin Basuki / 13516071

***

## Import

In [14]:
#Ini gw copas dari tucil dulu ya sementara, nanti yang ga butuh apus ae

import numpy as np 
import random
import pandas as pd 
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
import itertools

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn import tree
from sklearn.impute import SimpleImputer

from sklearn import svm, datasets
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, KFold

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from pprint import pprint

#Coba pake imputer


%matplotlib inline

## Read data

In [2]:
heartTrain = pd.read_csv('tubes2_HeartDisease_train.csv')
heartTest = pd.read_csv('tubes2_HeartDisease_test.csv')

## Analisis kondisi data

In [3]:
print('Tipe data pada kolom\n')
print(heartTrain.dtypes)
print()

print('Menampilkan sebagian data\n')
print(heartTrain.head())
print()

print('Mencari jumlah nilai yang tidak terdefinisi pada tiap kolom-nya')
heartTrain.isna().sum()

Tipe data pada kolom

Column1      int64
Column2      int64
Column3      int64
Column4     object
Column5     object
Column6     object
Column7     object
Column8     object
Column9     object
Column10    object
Column11    object
Column12    object
Column13    object
Column14     int64
dtype: object

Menampilkan sebagian data

   Column1  Column2  Column3 Column4 Column5 Column6 Column7 Column8 Column9  \
0       54        1        4     125     216       0       0     140       0   
1       55        1        4     158     217       0       0     110       1   
2       54        0        3     135     304       1       0     170       0   
3       48        0        3     120     195       0       0     125       0   
4       50        1        4     120       0       0       1     156       1   

  Column10 Column11 Column12 Column13  Column14  
0        0        ?        ?        ?         1  
1      2.5        2        ?        ?         1  
2        0        1        0        3  

Column1     0
Column2     0
Column3     0
Column4     0
Column5     0
Column6     0
Column7     1
Column8     0
Column9     0
Column10    0
Column11    0
Column12    0
Column13    0
Column14    0
dtype: int64

Berdasarkan dengan beberapa pengecekan di atas, dapat dilihat bahwa data pada csv yang diberikan:
1. Tidak semua data bertipe numerik
2. Ada beberapa data yang bernilai '?'
3. Ada data yang bernilai NaN (undefined)

Hal tersebut dapat mengganggu proses pemodelan. Oleh karena itu perlu dilakukan pre-processing sebagai berikut :

#### Konversi dataframe menjadi numerik

In [4]:
# Convert string to numeric, convert non-number to NAN
heartTrain = heartTrain.apply(pd.to_numeric,errors = 'coerce')

print('Tipe data pada kolom setelah konversi\n')
print(heartTrain.dtypes)
print()

# NaN count
print('Jumlah Nilai NaN setelah dataframe diconvert menjadi numerik\n')
print(heartTrain.isna().sum())

Tipe data pada kolom setelah konversi

Column1       int64
Column2       int64
Column3       int64
Column4     float64
Column5     float64
Column6     float64
Column7     float64
Column8     float64
Column9     float64
Column10    float64
Column11    float64
Column12    float64
Column13    float64
Column14      int64
dtype: object

Jumlah Nilai NaN setelah dataframe diconvert menjadi numerik

Column1       0
Column2       0
Column3       0
Column4      47
Column5      24
Column6      78
Column7       2
Column8      44
Column9      44
Column10     49
Column11    262
Column12    514
Column13    408
Column14      0
dtype: int64


#### Menghilangkan nilai NaN

Pada pre-processingnya, konversi data dari object (string) menjadi numerik berhasil menghilagkan tipe objek dari dataframe. Namun, untuk value yang tidak dapat dikonversi menjadi angka akan bernilai NaN yang membuat dataframe tidak bisa diolah. 

Salah satu cara termudah untuk menghilangkan nilai NaN adalah dengan cara menghapus row yang mengandung nilai tersebut, namun melihat pada column 12 terdapat 514 row yang bernilai NaN, cara ini tidak feasible karena akan sangat mengurangi data training. Oleh karena itum, kami memutuskan untuk me-replace nilai NaN dengan XXXXXX. Pemilihan XXXXXX dilakukan karena XXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [5]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')

c = heartTrain.columns
heartTrain = pd.DataFrame(imp.fit_transform(heartTrain))
heartTrain.columns = c
#imputed_DF.index = DF.index

#NaN count
print('Jumlah Nilai NaN setelah datafram di-imput\n')
print(heartTrain.isna().sum())


Jumlah Nilai NaN setelah datafram di-imput

Column1     0
Column2     0
Column3     0
Column4     0
Column5     0
Column6     0
Column7     0
Column8     0
Column9     0
Column10    0
Column11    0
Column12    0
Column13    0
Column14    0
dtype: int64


In [6]:
#Analisis buat milih algoritma(?)

In [7]:
# Percobaan coba convert yang diskrit ke interge
heartTrain[['Column1','Column2','Column3','Column6','Column7','Column9','Column11','Column12','Column13']] = heartTrain[['Column1','Column2','Column3','Column6','Column7','Column9','Column11','Column12','Column13']].astype(int)
heartTrain.dtypes
heartTrain.head(20)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14
0,54,1,4,125.0,216.0,0,0,140.0,0,0.0,2,0,6,1.0
1,55,1,4,158.0,217.0,0,0,110.0,1,2.5,2,0,6,1.0
2,54,0,3,135.0,304.0,1,0,170.0,0,0.0,1,0,3,0.0
3,48,0,3,120.0,195.0,0,0,125.0,0,0.0,2,0,6,0.0
4,50,1,4,120.0,0.0,0,1,156.0,1,0.0,1,0,6,3.0
5,64,0,4,130.0,303.0,0,0,122.0,0,2.0,2,2,3,0.0
6,63,1,4,130.0,308.0,0,0,138.0,1,2.0,2,0,6,2.0
7,58,1,2,130.0,251.0,0,0,110.0,0,0.0,2,0,6,0.0
8,42,1,2,150.0,268.0,0,0,136.0,0,0.0,2,0,6,0.0
9,54,1,3,120.0,258.0,0,2,147.0,0,4.0,2,0,7,0.0


In [8]:
#Split data train
X = heartTrain.drop('Column14', axis = 1)
Y = heartTrain['Column14']
KF = KFold(10, shuffle=True)

# for index, row in heartTrain.iterrows():
#     for c in heartTrain.columns:
#         if row[c] < 0:
#             print(index)
#             print(c)
#             print(row[c])

In [9]:
# Coba coba algoritma

## Algoritma Naive-Bayes

In [10]:
gnb = GaussianNB()

i=1
sum_acc = 0
sum_prec = 0
sum_rec = 0
for trainidx, testidx in KF.split(X):
    X_train, X_test = X.iloc[trainidx], X.iloc[testidx]
    Y_train, Y_test = Y.iloc[trainidx], Y.iloc[testidx]
    gnb.fit(X_train,Y_train)

    accuration = metrics.accuracy_score(Y_test, gnb.predict(X_test))
    precision = metrics.precision_score(Y_test, gnb.predict(X_test), average="macro")
    recall = metrics.recall_score(Y_test, gnb.predict(X_test), average="macro")
    
    i+=1
    sum_acc += accuration
    sum_prec += precision
    sum_rec += recall
    
print("Average Accuration : {0:.4f}".format(sum_acc/10))
print("Average Precision : {0:.4f}".format(sum_prec/10))
print("Average Recall : {0:.4f}".format(sum_rec/10))



Average Accuration : 0.5584
Average Precision : 0.3417
Average Recall : 0.3346


c:\users\asus rog\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## Algoritma KNN

In [11]:
knn = KNeighborsClassifier(n_neighbors=30)

i=1
sum_acc = 0
sum_prec = 0
sum_rec = 0
for trainidx, testidx in KF.split(X):
    X_train, X_test = X.iloc[trainidx], X.iloc[testidx]
    Y_train, Y_test = Y.iloc[trainidx], Y.iloc[testidx]
    knn.fit(X_train,Y_train)

    accuration = metrics.accuracy_score(Y_test, knn.predict(X_test))
    precision = metrics.precision_score(Y_test, knn.predict(X_test), average="macro")
    recall = metrics.recall_score(Y_test, knn.predict(X_test), average="macro")
    
#     print("Set ", i)
#     print("Accuration: {0:.4f}".format(accuration))
#     print("Precision: {0:.4f}".format(precision))
#     print("Recall: {0:.4f}".format(recall))
#     print()
    i+=1
    sum_acc += accuration
    sum_prec += precision
    sum_rec += recall
    
print("Average Accuration : {0:.4f}".format(sum_acc/10))
print("Average Precision : {0:.4f}".format(sum_prec/10))
print("Average Recall : {0:.4f}".format(sum_rec/10))

c:\users\asus rog\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\asus rog\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\asus rog\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\asus rog\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 

Average Accuration : 0.4929
Average Precision : 0.2854
Average Recall : 0.2704


c:\users\asus rog\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\asus rog\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Algoritma Decision Tree

In [12]:
tree_clf = tree.DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=5)
tree_clf.fit(X_train, Y_train)
tree_predict = tree_clf.predict(X_test)
print("Accuracy:", metrics.accuracy_score(Y_test, tree_predict))
print("Precision:", metrics.precision_score(Y_test, tree_predict, average='macro'))
print("Recall:", metrics.recall_score(Y_test, tree_predict, average='macro'))

Accuracy: 0.506493506494
Precision: 0.232126696833
Recall: 0.3


c:\users\asus rog\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Algoritma MLP 

In [15]:
mlp = MLPClassifier(hidden_layer_sizes=(26, 13), solver='lbfgs', max_iter=1000, learning_rate_init=0.00001)

results = cross_validate(mlp, X, Y, cv=10)
pprint(results)

print(sum(results['test_score'])/10)

{'fit_time': array([ 1.6179111 ,  0.07547498,  1.57005954,  1.57266188,  1.59270668,
        1.6314919 ,  1.52040696,  1.51095963,  1.65823531,  0.06498313]),
 'score_time': array([ 0.        ,  0.        ,  0.00099802,  0.00099754,  0.        ,
        0.00099087,  0.        ,  0.00213027,  0.        ,  0.        ]),
 'test_score': array([ 0.5625    ,  0.2375    ,  0.53164557,  0.57692308,  0.56410256,
        0.55844156,  0.62337662,  0.57142857,  0.49350649,  0.44736842]),
 'train_score': array([ 0.58082976,  0.2360515 ,  0.58285714,  0.58773181,  0.61768902,
        0.6011396 ,  0.61111111,  0.58119658,  0.59401709,  0.44807966])}
0.516679287845


In [ ]:
#Simpan model